In [79]:
import requests 
url = "https://www.nownews.com/cat/column/"

headers= {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url)
# print(res.text)

In [14]:
headers

{'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}

In [80]:
from bs4 import BeautifulSoup as bs
soup = bs(res.text,"lxml")
# print(soup)

In [10]:
### 抓取 1 ~ 10 篇文章連結
links = []

# 抓取 1~5
for ele in soup.select("div.sliderBlk a"):
#     print(ele)
    url = ele["href"]
#     print(url)
    links.append(url)
    
# 抓取 6~10
for ele in soup.select("div.leftCol div.listBlk a"):
#     print(ele)
    links.append(ele["href"])
    
for url in links:
    print(url)

https://www.nownews.com/news/5424941
https://www.nownews.com/news/5424915
https://www.nownews.com/news/5424737
https://www.nownews.com/news/5424908
https://www.nownews.com/news/5424900
https://www.nownews.com/news/5424663
https://www.nownews.com/news/5424901
https://www.nownews.com/news/5424615
https://www.nownews.com/news/5424853
https://www.nownews.com/news/5424811


In [38]:
### 抓取本文的 HTML 原始碼 (source code)
url2 = links[2]
res2 = requests.get(url2,headers=headers)
soup2 = bs(res2.text,"lxml")

# print(soup2)

In [47]:
### 抓取本文的 標題 , 分類 , 時間 , 內容

# 標題
title = soup2.select("div.titleBlk h1")[0].text
print("title:" , title)

print("-"*80)

# 分類
# \n -> 換行符號
category = soup2.select("div.breadCrumbBlk")[0].text
category = category.strip().replace("\n\n"," > ")
print("category :" , category)

print("-"*80)

# 時間
time = soup2.select("div.titleBlk p.time")[0].text.strip()
print("time :" , time)

print("-"*80)

# 內容
article = soup2.select("div.leftCol article")[0]
# print(article)

# 拔除 div.ad-blk1 , ul.related 標籤
if len(article.select("div.ad-blk1"))>0:
    article.select("div.ad-blk1")[0].extract()
# print(article.select("div.ad-blk1"))
# print(article.select("ul.related"))
if len(article.select("ul.related")) >0:
    article.select("ul.related")[0].extract()
    
# print(article)

content = article.text.strip()
print("content :",content)


title: 巷仔內／蔡英文證實美軍協訓國軍　台美默契加溫
--------------------------------------------------------------------------------
category : NOWnews 今日新聞 > 要聞 > 政治巷仔內
--------------------------------------------------------------------------------
time : 2021-10-28 17:53:22
--------------------------------------------------------------------------------
content : 蔡英文總統接受美國有線電視新聞網（CNN）獨家採訪，首度證實有美軍在台協助台灣軍隊訓練，並對美軍協防台灣有信心。這是我國政府高層首度證實美軍在台協訓部隊，也讓國際社會關心美國對台「戰略模糊」政策是否改變。蔡總統的專訪內容一出，立刻牽動美中台的大三角情勢，引發各界關注。蔡總統的說法明確而精準，勢必事先已和美方取得默契，由此可見，美國愈趨重視台灣這個盟友的角色，而未來美方是否會有進一步挺台的具體動作，值得持續觀察。

				            	蔡英文證實美軍在台協訓，部分媒體下標「美軍駐台」，引發立委質詢關切。對此，國防部長邱國正澄清，美軍不是部隊「進駐」台灣，而是協訓人員在台灣協助訓練國軍。軍隊「駐台」與「助台」僅有一字之差，但政治意涵差別相當大，依據國際法與國際慣例，一國部隊正式進駐，必須簽署軍事互助協定，軍人犯罪豁免條款等；協訓部隊不一定要現役軍人，退役與現役軍人都可以用顧問團名義來台，或是化身文職人員派駐在美國在台協會（AIT），前往營區與國軍交流。所謂「戰略模糊」（Strategic Ambiguity）是美國在台海戰爭危機上的模糊立場，不表明會如何應對共軍犯台，「戰略模糊」目的是不明講中國犯台美軍的回應措施，讓北京因為摸不著美軍反應而不敢採取軍事冒進行為，這是美國長期以來的對台政策。而「戰略清晰」（Strategic Clarity），則是美國明確表示中國犯台時的回應政策。隨著中國對台軍事威脅加深，川普總統執政末期，要求美國拋棄「戰略模糊」的聲勢不斷升溫。美國總統拜登日

In [66]:
### 抓取本文資料 -> 收集成一個 function 
def parsing_content_data(content_soup):
    
    # 標題
    title = content_soup.select("div.titleBlk h1")[0].text

    # 分類
    # \n -> 換行符號
    category = content_soup.select("div.breadCrumbBlk")[0].text
    category = category.strip().replace("\n\n"," > ")

    # 時間
    # time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    # 時間格式更新
    raw_time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    time = raw_time.split("｜")[0]

    # 內容
    article = content_soup.select("div.leftCol article")[0]

    # 拔除 div.ad-blk1 , ul.related 標籤
    if len(article.select("div.ad-blk1"))>0:
        article.select("div.ad-blk1")[0].extract()
    if len(article.select("ul.related")) >0:
        article.select("ul.related")[0].extract()

    content = article.text.strip()
    
    return {
        "title" : title,
        "category" : category,
        "time"  : time,
        "content" : content
    }


def get_source_code(url):
    headers= {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }
    res = requests.get(url,headers=headers)
    soup = bs(res.text,"lxml")
    
    return soup



url3 = links[8]
print(url3)
soup3 = get_source_code(url3)
parsing_content_data(soup3)

https://www.nownews.com/news/5424853


{'title': '越南11月擬開富國島旅遊！只認AZ、莫德納、BNT等9款疫苗',
 'category': 'NOWnews 今日新聞 > 全球 > 國際要聞',
 'time': '2021-10-28 17:07:52',
 'content': '越南近期公布最新承認的疫苗護照清單，一共採認9款疫苗，包括AZ、莫德納、輝瑞/BNT、嬌生、中國國藥、北京科興等，持這些疫苗護照的旅客，入境隔離天數可縮短至7天，而越南富國島也開始研擬自11月起將試點迎接疫苗護照清單中的國際觀光客，其中雖然台灣也暫列名單中，但施打國產高端疫苗的民眾，尚無法受惠於此政策。\n\r\n\t\t\t\t            \t根據越南通訊社（VNA）報導， 越南原計10月即開放安排接待國際觀光客，無奈當地疫苗接種率仍過低因此推延計畫，而越南政府22日晚間宣布，11月20起將安排接待，重迎國際觀光客，其中將由堅江省（Kien Giang）富國島打頭陣，會由新冠疫情較緩和、防控安全性高的國家與地區如歐洲、中東、東北亞、北美與澳洲等地區引進觀光客。若評估順利，接著從12月下旬至明年3月底，富國島將開始安排類似包機方式，接待5000名以上遊客。此外，當地媒體還報導，越南政府還想在11月開放聯合國教科文組織世界遺產下龍灣和會安古城，不過目前沒有詳細計畫。▲越南富國島11月20日起將重新迎接國際觀光客。（圖／美聯社／達志影像）這些前往富國島的國際觀光客除了得持有越南認可的疫苗接種證明、疫苗護照或康復證書外，還要在出發前3天出示PCR核酸檢PCR核酸檢測陰性證明，登記參加堅江省推薦的旅行社所推出的旅行團。目前越南僅暫時認可72個國家與地區的疫苗接種證明或疫苗護照，這些旅客入境集中隔離可縮短至7天，目前僅承認阿斯特捷利康（AstraZeneca，AZ）、嬌生（Johnson & Johnson）、莫德納（Moderna）、輝瑞/BNT（Pfizer/BioNTech）、俄羅斯衛星-V（Sputnik V）、中國國藥（Sinopharm）、Hayat-Vax、古巴阿布達拉（Abdala）以及北京科興（Sinovac）疫苗等9款疫苗。台灣雖也暫列名單中，但越南目前認可的疫苗廠牌僅限「世界衛生組織（WHO）認可，或美國疾病管制中心（CDC）、歐盟藥品管理局（EMA）批准，或已向越南政府申請註冊獲得核准

In [63]:
### 時間欄位修正
print(soup3.select("div.titleBlk p.time")[0])
raw_time = soup3.select("div.titleBlk p.time")[0].text.strip()
print(raw_time)
print(raw_time.split("｜"))
print(raw_time.split("｜")[0])

<p class="time">
<span class="i-calendar"></span>2021-10-28 17:46:28
				                    </p>
2021-10-28 17:46:28
['2021-10-28 17:46:28']
2021-10-28 17:46:28


In [68]:
### 抓取 1 ~ 10 篇文章本文
# url3 = links[8]
# print(url3)
dataList = []
for url in links:
    content_soup = get_source_code(url)
    data = parsing_content_data(content_soup)
    dataList.append(data)
    
    print("{} is ok.".format(url))
    
print("Done.")

https://www.nownews.com/news/5424941 is ok.
https://www.nownews.com/news/5424915 is ok.
https://www.nownews.com/news/5424737 is ok.
https://www.nownews.com/news/5424908 is ok.
https://www.nownews.com/news/5424900 is ok.
https://www.nownews.com/news/5424663 is ok.
https://www.nownews.com/news/5424901 is ok.
https://www.nownews.com/news/5424615 is ok.
https://www.nownews.com/news/5424853 is ok.
https://www.nownews.com/news/5424811 is ok.
Done.


In [69]:
dataList

[{'title': '軍公教調薪創25年新高\u3000人力銀行估：民間企業加薪上看5%',
  'category': 'NOWnews 今日新聞 > 要聞',
  'time': '2021-10-28 18:14:16',
  'content': '行政院今（28）日宣布，軍公教明（2022）年調薪4%，創下25年來最高，不過104人力銀行資深副總暨人資長鍾文雄接受《NOWnews今日新聞》採訪表示，中央帶頭調薪也帶動民間企業明年的調薪幅度，再加上GDP成長創下10年來新高，整體看來，明年度的企業調薪幅度能達到3.5%至5%，但仍受到產業別而有所不同。\n\r\n\t\t\t\t            \t行政院表示，考量國內肺炎疫情趨緩，且民間企業平均薪資及基本工資並已有相當成長，決定2022年度軍公教人員待遇調升4％，現職人員待遇調整所需經費314億元，包括中央政府153億元及地方政府161億元。鍾文雄指出，過去軍公教有調薪都是3%左右，以104玩數據所做的調查顯示，民間企業過去的調薪幅度大概也只有調升2.33%左右，不過由於物價指數衝高，GDP成長6.7%，人均所得有望突破3萬美元，都帶動整體薪資上漲，上調幅度有望來到3.5%至5%。以104人力銀行內部的的調薪率為例，鍾文雄說明，往年約在3.3%至3.5%左右，以今年的預測來說，可能有3.5%至5%，不過以整體看來，能有5%的產業，大部分是在搶才的積極性比較高的資訊、半導體產業，一來是他們的營運的支付能力比較高，二來也是整體需求大增直接影響調薪幅度。然而，對於本身就相對低薪的餐飲服務、飯店產業的薪資調幅可能依然低靡。鍾文雄說明，以Q4來看雖然內需大力回彈，但因為蕭條時間太長，現在的報復性消費也僅能讓產業回血，基層服務業恐維持低薪現況。'},
 {'title': '冬奧剩不到百天！中國新一輪疫情燒\u3000北京死守、東北警戒',
  'category': 'NOWnews 今日新聞 > 全球 > 國際要聞',
  'time': '2021-10-28 17:56:22',
  'content': '中國近日本土疫情升溫，27日再度新增23例本土病例，以甘肅與內蒙古居多，首都北京市則有3例。近日疫情蔓延至北京，根據了解北京在這輪疫情共存在3個獨立傳播分支。不過連東北地區加強防疫措施，限制旅遊和

In [78]:
### 資料輸出成檔案 (資料落地)
# data = dataList[0]
# print(data)

for data in dataList:

    file_name = "sample/{}.txt".format(data["title"])
    with open(file_name,"w",encoding="utf8") as out_file:
        
        record = ""
        
        # 資料全部讀取完 , 再一次寫出檔案
        for key in data:
            record += "{}:{}\n".format(key , data[key])
            
        out_file.write(record)

        # 每讀一段資料 , 就寫出檔案一次 (不佳)
        # for key in data:
        #     msg = "{}:{}\n".format(key , data[key])
        #     out_file.write(msg)
    

In [84]:
### 抓取更多文章 
# ulNewsList
# len(soup.select("#ulNewsList a"))

for ele in soup.select("#ulNewsList a"):
    print(ele["href"])

https://www.nownews.com/news/5424985
https://www.nownews.com/news/5424982
https://www.nownews.com/news/5424983
https://www.nownews.com/news/5424941
https://www.nownews.com/news/5424915
https://www.nownews.com/news/5424737
https://www.nownews.com/news/5424908
https://www.nownews.com/news/5424900


In [94]:
### 透過 API 抓 JSON 資料

# 1. 抓取 txtPageNo , 取得 1st 參數
pid = soup.select("input#txtPageNo")[0]["value"]
print(pid)

# 2. 向 API 發 requests , 取得文章列表資料
api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}".format(pid)
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(api,headers=headers)

# response 用文字格式輸出
# print(res.text)

# response 用 json 格式輸出
data = res.json()
# print(data)

# 3. 再透過 API 的資料 , 取得下一次的 requests 參數

5424900


In [93]:
for ele in data["data"]["newsList"]:
    print(ele["postTitle"])
    # print(ele["postUrl"])
    
    url = "https://www.nownews.com" + ele["postUrl"]
    print(url)

    print("-"*80)

布林肯挺台加入聯合國體系　學者：應爭取參加周邊組織
https://www.nownews.com/news/column/5424663
--------------------------------------------------------------------------------
《寂寞星球》10大旅遊城市出爐！台北「奪下亞軍」原因曝
https://www.nownews.com/news/column/5424901
--------------------------------------------------------------------------------
蔡英文證實美軍協訓國軍　學者：戰略轉向建設性清晰
https://www.nownews.com/news/column/5424615
--------------------------------------------------------------------------------
越南11月擬開富國島旅遊！只認AZ、莫德納、BNT等9款疫苗
https://www.nownews.com/news/column/5424853
--------------------------------------------------------------------------------
新竹光臨藝術節來了！原住民、新住民與客家裝置藝術曝
https://www.nownews.com/news/column/5424811
--------------------------------------------------------------------------------
首場公投說明會辦在校園挨批　民進黨改至三民運動公園
https://www.nownews.com/news/column/5424808
--------------------------------------------------------------------------------
移工宿舍私設充電樁有安全風險？台電：勸導不聽會斷電
https://www.nownews.com/news/column/5424809
------------------

In [ ]:
# 3. 再透過 API 的資料 , 取得下一次的 requests 參數